We will work today with 'momentum_data.xlsx' file. It contains several tabs:

- The 1st tab contains the momentum factor as an excess return: $\tilde{r}^{\text{mom}}$.

- The 3rd tab contains returns on portfolios corresponding to scored momentum deciles.

    - $r^{\text{mom(1)}}$ denotes the portfolio of stocks in the lowest momentum decile, the 'losers' with the lowest past returns.
    
    - $r^{\text{mom(10)}}$ denotes the portfolio of stocks in the highest momentum decile.
    
- The 4th tab gives portfolios sorted by momentum and size.

    - $r^{\text{momSU}}$ denotes the portfolio of small stocks in the top 3 deciles of momentum scores.
    
    - $r^{\text{momBD}}$ denotes the portfolio of big-stocks in the bottom 3 deciles of momentum scores.

In [2]:
import pandas as pd
from datetime import datetime

In [9]:
factors = pd.read_excel("momentum_data.xlsx", "factors (excess returns)", index_col=0)
momentum = pd.read_excel("momentum_data.xlsx", "momentum (excess returns)", index_col=0)
size_sorts = pd.read_excel("momentum_data.xlsx", "size_sorts (total returns)", index_col=0)
risk = pd.read_excel("momentum_data.xlsx", "risk-free rate", index_col=0)
decil = pd.read_excel("momentum_data.xlsx", "deciles (total returns)", index_col=0)

### Task 1.

Check that momentum return, $\tilde{r}^{\text{mom:FF}}$, given in the second tab, is constructed:

$\tilde{r}^{\text{mom:FF}} = (r^{\text{momBU}}+r^{\text{momSU}})/2-
(r^{\text{momBD}}+r^{\text{momSd}})/2$

In [4]:
r_mom_FF = pd.DataFrame(index=momentum.index)
r_mom_FF["UMD 1"] = round(momentum["UMD"], 1)
r_mom_FF["UMD 2"] = round((size_sorts["BIG HiPRIOR"] + size_sorts["SMALL HiPRIOR"])/2 - (size_sorts["BIG LoPRIOR"] + size_sorts["SMALL LoPRIOR"])/2, 1)
all(r_mom_FF["UMD 1"] == r_mom_FF["UMD 2"])

True

### Task 2

Fill in next table 1 with the appropriate stats for $\tilde{r}^{\text{mom:FF}}$. Please note, that $\tilde{r}^m$ is the excess market return, MKT; $\tilde{r}^v$ is the value portfolio, HML.

<img src='table1.png'>

In [5]:
momentum_per = pd.DataFrame(columns = ["Mean", "Vol", "Sharpe", "Corr_to_Rn", "Corr_to_Rv"]).rename_axis('Subsample')
years = ["1927-2022", "1927-1993", "1994-2008", "2009-2022"]
for year in years:
    momentum_per.loc[year, "Mean"] = r_mom_FF.loc[year[:4]:year[-4:], "UMD 1"].mean()
    momentum_per.loc[year, "Vol"] = r_mom_FF.loc[year[:4]:year[-4:], "UMD 1"].std()
    momentum_per.loc[year, "Sharpe"] = momentum_per.loc[year, "Mean"] / momentum_per.loc[year, "Vol"]
    momentum_per.loc[year, "Skewness"] = r_mom_FF.loc[year[:4]:year[-4:], "UMD 1"].skew()
    momentum_per.loc[year, "Corr_to_Rm"] = r_mom_FF.loc[year[:4]:year[-4:], "UMD 1"].corr(factors.loc[year[:4]:year[-4:], "MKT"])
    momentum_per.loc[year, "Corr_to_Rv"] = r_mom_FF.loc[year[:4]:year[-4:], "UMD 1"].corr(factors.loc[year[:4]:year[-4:], "HML"])
momentum_per

,Mean,Vol,Sharpe,Corr_to_Rn,Corr_to_Rv,Skewness,Corr_to_Rm
Subsample,,,,,,,
1927-2022,0.000609,0.049822,0.012228,NaN,-0.405482,-2.188826,-0.345801
1927-1993,0.000871,0.047991,0.018142,NaN,-0.489162,-3.025223,-0.371853
1994-2008,0.003889,0.060134,0.064671,NaN,-0.109297,-0.482249,-0.250653
2009-2022,-0.004242,0.046,-0.092226,NaN,-0.399363,-1.684882,-0.347623


### Task 3

Construct your own long-only implementation:

$\tilde{r}^{\text{momU:FF}} = (r^{\text{momBU}}+r^{\text{momSU}})/2 - r^f.$

Note that you have to subtract the risk-free rate (RF) to get the excess return of this portfolio.

Fill out Table 2 for the data in the period 1994-2022.

<img src='table2.png'>

In [6]:
r_mom = pd.DataFrame()
r_mom["Long-and-short, (r_mom:FF)"] = momentum["UMD"]
r_mom["Long-only, (r_momU:FF)"] = (size_sorts["BIG HiPRIOR"] + size_sorts["SMALL HiPRIOR"])/2 - risk["RF"]

long_only_imp = pd.DataFrame(columns=["mean", "vol", "Sharpe", "skew", 
                                      "Corr. to r~m", "Corr. to r~v"]).rename_axis("1994-2022")
for i in r_mom.columns:
    r_mom_temp = r_mom.loc["1994":, i]
    long_only_imp.loc[i] = [r_mom_temp.mean(), r_mom_temp.std(), r_mom_temp.mean()/r_mom_temp.std(),
                            r_mom_temp.skew(), r_mom_temp.corr(factors["MKT"]), r_mom_temp.corr(factors["HML"])]

long_only_imp

,mean,vol,Sharpe,skew,Corr. to r~m,Corr. to r~v
1994-2022,,,,,,
"Long-and-short, (r_mom:FF)",0.003946,0.048672,0.081075,-1.433554,-0.306139,-0.238601
"Long-only, (r_momU:FF)",0.009399,0.051645,0.181984,-0.482872,0.903499,-0.154667


### Task 4

Assess how sensitive the threshold for the 'winners' and 'losers' is in the results. Specifically, we compare three constructions:

- long the top 1 decile and short the bottom 1 deciles:

$\tilde{r}^{\text{momD1}} = r^{\text{mom(10)}}-r^{\text{mom(1)}}$

• long the top 3 deciles and short the bottom 3 deciles:

$\tilde{r}^{\text{momD3}} = 
(r^{\text{mom(8)}}+r^{\text{mom(9)}}+r^{\text{mom(10)}})/3-
(r^{\text{mom(1)}}+r^{\text{mom(2)}}+r^{\text{mom(3)}})/3 = 
\frac 13 \sum^{10}_{k=8} r^{\text{mom(k)}} + 
\frac 13 \sum^{3}_{k=1} r^{\text{mom(k)}}$
 

• long the top 5 deciles and short the bottom 5 deciles:

$\tilde{r}^{\text{momD5}} = 
\frac 15 \sum^{10}_{k=6} r^{\text{mom(k)}} + 
\frac 15 \sum^{5}_{k=1} r^{\text{mom(k)}}$

Compare all three constructions by filling out the stats in the table 3 below for the period 1994-2022.

<img src='table3.png'>

In [7]:
r_mom_D = pd.DataFrame()
r_mom_D["r_mom_D1"] = decil.iloc[:, 9] - decil.iloc[:, 0]
r_mom_D["r_mom_D2"] = sum([1/3*(decil.iloc[:, -1 -i] - decil.iloc[:, i]) for i in range(3)])
r_mom_D["r_mom_D5"] = sum([1/5*(decil.iloc[:, -1 -i] - decil.iloc[:, i]) for i in range(5)])

momentum = pd.DataFrame(columns=["mean", "vol", "Sharpe", "skew", 
                                      "Corr. to r~m", "Corr. to r~v"]).rename_axis("1994-2022")

for i in r_mom_D.columns:
    r_mom_D_temp = r_mom_D.loc["1994", i]
    momentum.loc[i] = [r_mom_D_temp.mean(), r_mom_D_temp.std(), r_mom_D_temp.mean()/r_mom_D_temp.std(),
                           r_mom_D_temp.skew(), r_mom_D_temp.corr(factors["MKT"]), r_mom_D_temp.corr(factors["HML"])]
momentum

,mean,vol,Sharpe,skew,Corr. to r~m,Corr. to r~v
1994-2022,,,,,,
r_mom_D1,0.002867,0.025169,0.113896,0.025363,0.293731,-0.534652
r_mom_D2,-0.000492,0.014006,-0.035105,0.562636,0.378111,-0.612532
r_mom_D5,0.001157,0.010005,0.115607,0.919455,0.496886,-0.614538


### Task 5
Use the data provided on both small-stock 'winners', $r^{\text{momSU}}$, and small-stock 'losers', $r^{\text{momSD}}$
to construct a small-stock momentum portfolio,

$
r_t^{\text{momS}} = r^{\text{momSU}} - r^{\text{momSD}}
$

Similarly, use the data provided to construct a big-stock momentum portfolio,

$
r_t^{\text{momB}} = r^{\text{momBU}} - r^{\text{momBD}}
$

Fill out Table 4 over the sample 1994-2022.

<img src='table4.png'>

In [10]:
r_mom_st = pd.DataFrame()
r_mom_st["All stocks, r~mom:FF"] = momentum["UMD"]
r_mom_st["Small stocks, rtmomS"] = size_sorts["SMALL HiPRIOR"] - size_sorts["SMALL LoPRIOR"]
r_mom_st["Large stocks, rtmomB"] = size_sorts["BIG HiPRIOR"] - size_sorts["BIG LoPRIOR"]

sb_momen = pd.DataFrame(columns=["mean", "vol", "Sharpe", "skewness", 
                                 "Corr. to r~[m]"]).rename_axis('1994-2022')

for i in r_mom_st.columns:
    r_mom_st_temp = r_mom_st.loc["1994", i]
    sb_momen.loc[i] = [r_mom_st_temp.mean(), r_mom_st_temp.std(), r_mom_st_temp.mean()/r_mom_st_temp.std(),
                      r_mom_st_temp.skew(), r_mom_st_temp.corr(factors["MKT"])]
    

sb_momen

,mean,vol,Sharpe,skewness,Corr. to r~[m]
1994-2022,,,,,
"All stocks, r~mom:FF",0.002992,0.015030,0.199051,0.502030,0.390406
"Small stocks, rtmomS",0.008517,0.022222,0.383249,0.471552,0.252770
"Large stocks, rtmomB",-0.002542,0.013603,-0.186842,0.686852,0.453348


### Task 6

Re-do Tables 2 and 4 but for the size portfolios of the Value factor. Get this data from Ken French's website, https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

Specifically, see the following data set:

- Size-sorted portfolios for value. "6 Portfolios Formed on Size and Book-to-Market (2 x 3)"

In [11]:
vf = pd.read_csv("6_Portfolios_2x3.csv",index_col=0, skip_blank_lines=False,
                           header=15, na_values=[-99.99, -999], nrows=1155)

vf.index = vf.index.astype("str").map(lambda st: datetime.strptime(st, "%Y%m")).to_period().to_timestamp("M")

vf = vf.loc["1994":]

r_mom_vf = pd.DataFrame()

r_mom_vf["Long-and-short"] = (vf["BIG HiBM"] + vf["SMALL HiBM"])/2 - (vf["BIG LoBM"] + vf["SMALL LoBM"])/2

r_mom_vf["Long-only"] = (vf["BIG HiBM"] + vf["SMALL HiBM"]) / 2 - risk["RF"].loc["1994":]

long_vf = pd.DataFrame(columns=["mean", "vol", "Sharpe", "skew", 
                                 "Corr. to r~m", "Corr. to r~v"]).rename_axis("1994-2022")
for i in r_mom_vf.columns:
    sub_mom = r_mom_vf.loc["1994":, i]
    long_vf.loc[i] = [sub_mom.mean(), sub_mom.std(), sub_mom.mean()/sub_mom.std(),
                         sub_mom.skew(), sub_mom.corr(factors["MKT"]), sub_mom.corr(factors["HML"])]

r_mom_SB_vf = pd.DataFrame()
r_mom_SB_vf["All stocks"] = r_mom_vf.iloc[:, 0]
r_mom_SB_vf["Small stocks"] = vf["SMALL HiBM"] - vf["SMALL LoBM"]
r_mom_SB_vf["Large stocks"] = vf["BIG HiBM"] - vf["BIG LoBM"]

sb_stock_vf = pd.DataFrame(columns=["mean", "vol", "Sharpe", "skew", 
                                    "Corr. to r~m", "Corr. to r~v"]).rename_axis("1994-2022")

for i in r_mom_SB_vf.columns:
    sub_mom = r_mom_SB_vf.loc[:, i]
    sb_stock_vf.loc[i] = [sub_mom.mean(), sub_mom.std(), sub_mom.mean()/sub_mom.std(),
                          sub_mom.skew(), sub_mom.corr(factors["MKT"]), sub_mom.corr(factors["HML"])]
    
    
print(long_vf)
sb_stock_vf

                    mean       vol    Sharpe      skew  Corr. to r~m  \
1994-2022                                                              
Long-and-short  0.132581  3.315845  0.039984  0.207751     -0.102250   
Long-only       0.998077  5.495839  0.181606 -0.908702      0.866864   

                Corr. to r~v  
1994-2022                     
Long-and-short      1.000000  
Long-only           0.330352  


,mean,vol,Sharpe,skew,Corr. to r~m,Corr. to r~v
1994-2022,,,,,,
All stocks,0.132581,3.315845,0.039984,0.207751,-0.102250,1.000000
Small stocks,0.306017,3.750472,0.081594,0.271015,-0.269082,0.900306
Large stocks,-0.040855,3.641729,-0.011219,0.015334,0.090916,0.893837


### Task 7

Re-do Table 3 for the decile portfolios of the Value factor. Get this data from Ken French's website, https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

Specifically, see the following data set:

- "Portfolios Formed on Book-to-Market"

In [12]:
vff = pd.read_csv("Portfolios_Formed_on_BE-ME.csv", index_col=0, skip_blank_lines=False,
                  header=23, na_values=[-99.99, -999], nrows=1155)

vff.index = vff.index.astype("str").map(lambda st: datetime.strptime(st, "%Y%m")).to_period().to_timestamp("M")

vff = vff.loc["1994":].iloc[:, -10:]

r_mom_vff = pd.DataFrame()
for i in [1, 3, 5]:
    r_mom_vff[f"D{i}"] = sum([1/i*(vff.iloc[:, -j - 1] - vff.iloc[:, j]) for j in range(i)])

vff_mom = pd.DataFrame(columns = ["mean", "vol", "Sharpe", "skew", 
                                  "Corr. to r~m", "Corr. to r~v"]).rename_axis("1994-2022")

for i in r_mom_vff.columns:
    sub_mom = r_mom_vff.loc["1994":, i]
    vff_mom.loc[i] = [sub_mom.mean(), sub_mom.std(), sub_mom.mean() / sub_mom.std(),
                      sub_mom.skew(), sub_mom.corr(factors["MKT"]), sub_mom.corr(factors["HML"])]

    
vff_mom

,mean,vol,Sharpe,skew,Corr. to r~m,Corr. to r~v
1994-2022,,,,,,
D1,0.098754,5.106326,0.019339,-0.152904,0.178362,0.758798
D3,0.048029,3.350795,0.014334,-0.289519,0.174831,0.819238
D5,0.001467,2.373432,0.000618,-0.039878,0.158168,0.831363
